In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu, ttest_ind
from statsmodels.stats.multitest import multipletests


# GPT 3.5 - Load all isolated results

In [9]:
data_path = Path('data')

# find files
files = list(data_path.glob('results_scenario*gpt-3.5*.csv'))

df_single = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)



In [10]:
df_single

,gen_id,answer,mfv_code,model
0,1,3,205,gpt-3.5-turbo
1,2,4,205,gpt-3.5-turbo
2,3,4,205,gpt-3.5-turbo
3,5,3,205,gpt-3.5-turbo
4,7,4,205,gpt-3.5-turbo
...,...,...,...,...
1890,102,3,111,gpt-3.5-turbo
1891,104,3,111,gpt-3.5-turbo
1892,105,3,111,gpt-3.5-turbo
1893,106,3,111,gpt-3.5-turbo


Checking correct-answer effect

In [11]:
df_single.groupby("mfv_code")["answer"].nunique()

mfv_code
105    4
111    4
201    3
203    3
205    3
206    3
408    3
409    3
501    5
503    3
509    4
602    5
610    4
702    5
704    5
808    5
Name: answer, dtype: int64

## Loads general data

In [12]:
df_in_series = pd.read_csv("data/results_original_gpt-3.5-turbo_2023-08-13_22-43.csv")

df_in_series.head(2)

,102,103,104,105,108,109,110,111,112,113,...,714,715,716,801,802,803,804,805,808,810
0,4,4,3,4,4,3,4,3,4,3,...,5,4,5,5,3,5,4,3,3,3
1,4,3,3,4,4,2,2,3,4,2,...,5,4,5,5,4,5,5,4,4,4


Run tests

In [14]:
def cohen_d(x,y):
        from math import sqrt
        return (np.mean(x) - np.mean(y)) / sqrt((np.std(x, ddof=1) ** 2 + np.std(y, ddof=1) ** 2) / 2.0)


In [22]:
def compare_single_series(df_single, df_in_series):
    import scipy.stats
    test_results = list()

    for code in df_single["mfv_code"].unique():
        single_answers = df_single[df_single["mfv_code"] == code]["answer"]

        series_answers = df_in_series[str(code)]
        print(
            f"Code: {code} had {df_in_series[str(code)].nunique()} different answers when generated for all vignettes"
        )

        test_results.append(
            (
                code,
                np.median(single_answers),
                scipy.stats.median_abs_deviation(single_answers),
                np.median(series_answers),
                scipy.stats.median_abs_deviation(series_answers),
                cohen_d(single_answers, series_answers),
                *mannwhitneyu(single_answers, series_answers),
            )
        )

    df_tests = pd.DataFrame(
        test_results,
        columns=["mfv_code", "mediana_single", "MAD_single", "mediana_serial", "MAD_serial", "cohen_d", "u", "p-value"],
    )
    df_tests["rejected"], df_tests["adjusted_p-value"], _, _ = multipletests(
        df_tests["p-value"],
        alpha=0.05,
        method="bonferroni"
    )
    # round adjusted p-value to 3 decimals
    df_tests["adjusted_p-value"] = df_tests["adjusted_p-value"].apply(lambda x: round(x, 4))

    df_tests["median_dif"] = df_tests["mediana_single"] - df_tests["mediana_serial"]
    return df_tests

In [23]:
df_single.groupby("mfv_code").size()

mfv_code
105    106
111    179
201    100
203     99
205     87
206     96
408     90
409     97
501    100
503     99
509    202
602    100
610     91
702    174
704    103
808    172
dtype: int64

In [24]:
tests_gpt_35 = compare_single_series(df_single, df_in_series)

tests_gpt_35

Code: 205 had 3 different answers when generated for all vignettes
Code: 111 had 3 different answers when generated for all vignettes
Code: 203 had 2 different answers when generated for all vignettes
Code: 105 had 4 different answers when generated for all vignettes
Code: 201 had 2 different answers when generated for all vignettes
Code: 602 had 3 different answers when generated for all vignettes
Code: 409 had 4 different answers when generated for all vignettes
Code: 808 had 5 different answers when generated for all vignettes
Code: 501 had 4 different answers when generated for all vignettes
Code: 509 had 4 different answers when generated for all vignettes
Code: 704 had 5 different answers when generated for all vignettes
Code: 702 had 4 different answers when generated for all vignettes
Code: 610 had 4 different answers when generated for all vignettes
Code: 503 had 3 different answers when generated for all vignettes
Code: 408 had 4 different answers when generated for all vigne

,mfv_code,mediana_single,MAD_single,mediana_serial,MAD_serial,cohen_d,u,p-value,rejected,adjusted_p-value,median_dif
0,205,4.0,0.0,4.0,0.0,-0.703878,3732.5,3.203127e-06,True,0.0001,0.0
1,111,3.0,0.0,3.0,0.0,0.890690,15465.0,5.027970e-13,True,0.0000,0.0
2,203,4.0,0.0,5.0,0.0,-1.837652,1917.0,7.403682e-28,True,0.0000,-1.0
3,105,4.0,0.0,4.0,0.0,-0.264728,5709.5,2.445954e-02,False,0.3914,0.0
4,201,5.0,0.0,5.0,0.0,-0.727063,4772.0,1.569465e-08,True,0.0000,0.0
5,602,3.0,0.0,3.0,0.0,-0.368304,5279.5,6.178656e-03,False,0.0989,0.0
6,409,4.0,0.0,4.0,0.0,-0.599308,4305.5,1.046135e-05,True,0.0002,0.0
7,808,4.0,1.0,4.0,1.0,-0.086182,10382.5,5.125071e-01,False,1.0000,0.0
8,501,3.0,0.0,3.0,0.0,0.171528,6662.5,3.511010e-01,False,1.0000,0.0
9,509,3.0,0.0,4.0,0.0,-2.241929,2136.0,4.309372e-46,True,0.0000,-1.0


# GPT-4

In [25]:
data_path = Path('data')

# find files
files = list(data_path.glob('results_scenario*gpt-4*.csv'))

single_gpt4 = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)



Checking correct-answer effect

In [26]:
single_gpt4["mfv_code"].value_counts()

501    108
610    108
808    108
704    108
111    108
203    108
602    108
509    108
409    108
206    108
702    108
205    108
201    108
503    108
408    108
105    108
Name: mfv_code, dtype: int64

In [27]:
series_gpt4 = pd.concat(
    map(pd.read_csv, data_path.glob("results_original_gpt-4*.csv"),)
)

series_gpt4.head(2)

,102,103,104,105,108,109,110,111,112,113,...,714,715,716,801,802,803,804,805,808,810
0,4,4,4,5,5,3,5,4,5,3,...,2,2,4,5,4,5,5,4,4,4
1,4,4,4,5,5,3,5,3,5,4,...,5,3,4,5,5,5,5,4,5,5


In [29]:
tests_gpt4 = compare_single_series(single_gpt4, series_gpt4)

tests_gpt4

Code: 501 had 3 different answers when generated for all vignettes
Code: 610 had 5 different answers when generated for all vignettes
Code: 808 had 2 different answers when generated for all vignettes
Code: 704 had 4 different answers when generated for all vignettes
Code: 111 had 4 different answers when generated for all vignettes
Code: 203 had 1 different answers when generated for all vignettes
Code: 602 had 3 different answers when generated for all vignettes
Code: 509 had 3 different answers when generated for all vignettes
Code: 409 had 3 different answers when generated for all vignettes
Code: 206 had 2 different answers when generated for all vignettes
Code: 702 had 5 different answers when generated for all vignettes
Code: 205 had 2 different answers when generated for all vignettes
Code: 201 had 1 different answers when generated for all vignettes
Code: 503 had 3 different answers when generated for all vignettes
Code: 408 had 4 different answers when generated for all vigne

/var/folders/mj/j3hv11_n2k52jb10d4y2k5d00000gn/T/ipykernel_47112/3504218558.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return (np.mean(x) - np.mean(y)) / sqrt((np.std(x, ddof=1) ** 2 + np.std(y, ddof=1) ** 2) / 2.0)


,mfv_code,mediana_single,MAD_single,mediana_serial,MAD_serial,cohen_d,u,p-value,rejected,adjusted_p-value,median_dif
0,501,3.0,0.0,4.5,0.5,-3.842984,162.0,3.914587e-45,True,0.0000,-1.5
1,610,3.0,0.0,4.0,1.0,-1.116350,3618.0,3.004363e-16,True,0.0000,-1.0
2,808,5.0,0.0,5.0,0.0,0.328221,7722.0,1.711556e-02,False,0.2738,0.0
3,704,3.0,0.0,4.0,1.0,-1.312522,3132.0,1.348824e-18,True,0.0000,-1.0
4,111,3.0,0.0,4.0,1.0,-1.170618,3726.0,4.728379e-16,True,0.0000,-1.0
5,203,5.0,0.0,5.0,0.0,NaN,7344.0,1.000000e+00,False,1.0000,0.0
6,602,3.0,0.0,4.0,0.0,-3.500006,270.0,4.259920e-44,True,0.0000,-1.0
7,509,4.0,0.0,5.0,0.0,-0.984179,3942.0,1.914391e-13,True,0.0000,-1.0
8,409,4.0,0.0,5.0,0.0,-2.447448,1620.0,1.230372e-33,True,0.0000,-1.0
9,206,5.0,0.0,5.0,0.0,0.635678,8586.0,7.478582e-06,True,0.0001,0.0
